This notebook contains the code used create a stitch the strain maps for the manuscript: **"High-Resolution Digital Image Correlation study of the strain localization in a shot-peened RR1000 nickel-based superalloy"** by Alberto Orozco-Caballero, Thomas Jackson and João Quinta da Fonseca. 

### Import modules

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import HRDIC_analysis as hrdic
from ipywidgets import interact, interactive, fixed
from skimage import io
from skimage import color
from glob import glob

### Functions

Functions for loading displacement data, calculating strain and stitching maps together. It also masks out sources of error like dust so they are not interpreted as strain values. Masks were created manualy for each map using a raster graphics editor and are provided as bitmaps along with the data. 

In [3]:
def load_data(data_dir, steps):
    '''Function for loading in text files, masking them and 
        saving as numpy arrays'''
    #load data and masks
    for step in steps:
        def_maps = []
        masks=[]
        for i in range(6):
            def_maps.append(hrdic.DeformationMap(data_dir, str(step)+'_{:d}.txt'.format(i+1)))
            masks.append(io.imread('../data/mask-{:d}.tif'.format(i+1)))
        
        masked_max_shear_list=[]
        masked_axial_list=[]
        # use masks to remove noisy points
        for def_map, mask in zip(def_maps,masks):
            shear_map=def_map.max_shear
            axial_map=def_map.f11
            mask=color.rgb2gray(mask)
            shear_map[mask != 1] = -999
            axial_map[mask != 1] = -999
            masked_max_shear_list.append(shear_map)
            masked_axial_list.append(axial_map)
        np.savez('shear_'+str(step), 
                 masked_max_shear_list[0], masked_max_shear_list[1],
                 masked_max_shear_list[2], masked_max_shear_list[3],
                 masked_max_shear_list[4], masked_max_shear_list[5])
        np.savez('axial_'+str(step), 
                 masked_axial_list[0], masked_axial_list[1],
                 masked_axial_list[2], masked_axial_list[3],
                 masked_axial_list[4], masked_axial_list[5])
    
def stitch_maps(masked_strain_files,crop_list):
    ''' Function for stitching strain maps into one big map'''
    masked_strain_list=[]
    masked_strains=np.load(masked_strain_files)
    for array, strain in sorted(masked_strains.items()):
        masked_strain_list.append(strain)
        
    cropped_maps=[]
    for crop,def_map in zip(crop_list,masked_strain_list):
        cropped=def_map[:,crop[0]:crop[1]]
        cropped_maps.append(cropped)    
    top=np.hstack((cropped_maps[0],cropped_maps[1],cropped_maps[2]))
    bottom=np.hstack((cropped_maps[3],cropped_maps[4],cropped_maps[5]))
    #crop images and stack verically
    crop_lines=(612,59)
    all_height=np.vstack((top[:crop_lines[0],:],bottom[crop_lines[1]:,:]))
    return all_height

These large maps were obtained in sections and the cell below can be used to convert these into more efficient, compressed `numpy` arrays containing strain values. It's made available should the numpy arrays (also provided) become difficult to read or a different strain calculation is wanted. 

In [4]:
#Only reload data if needed
#data_dir='../data/'
#load_data(data_dir,range(1,7))

### Shear strains:

Before stitching the images need to be cropped. This is the definitive list of crop coordinates used for each map.

In [5]:
crop_list= [(26,-244),(105,-275),(301,-294),
            (25,-240),(110,-275),(300,-295)]

Stitch maps and save in compressed numpy array containing the a dictionary with strain maps at all deformation levels.

In [6]:
stitched_map_list=[]
shear_files=glob('shear_*.npz')
for strain_file in sorted(shear_files):
    stitched_map=stitch_maps(strain_file, crop_list)
    stitched_map_list.append(stitched_map)

Save maps.

In [8]:
np.save('stitched_shear_maps',stitched_map_list)